# Prerequisites

install promptflow-sdk and other dependencies:

In [ ]:
# install needed packages
!pip install -r requirements.txt

write environment variables in .env file:

In [ ]:
%%writefile .env
CHAT_DEPLOYMENT_NAME=gpt-35-turbo
AZURE_OPENAI_API_KEY=
AZURE_OPENAI_API_BASE=https://wanhanopenai.openai.azure.com/

# Run promptflow

In [ ]:
# Step 1. init flow directory - create promptflow folder from existing python file
!pf init . --entry intent.py --function extract_intent --prompt-template user_prompt_template=user_intent_zero_shot.md

In [ ]:
# Step 2. run with single line input
!pf test --input ./data/denormalized-flat.jsonl --env .env

In [ ]:
# Step 3. bulk run with multiple lines input
!pf run_bulk --input ./data/denormalized-flat.jsonl --output ./outputs/ --env .env

In [ ]:
# Step 4. evaluation
!pf eval --flow ../classification_accuracy_evaluation --input ./data/denormalized-flat.jsonl --bulk-run-output ./outputs/ --eval-output ./outputs/eval_output.jsonl --column-mapping "groundtruth=data.intent,prediction=variants.output.output"

In [ ]:
# Step 5. visualize
!pf visualize ./outputs/eval_output.jsonl 

# Run promptflow in cloud

In [ ]:
# Define the workspace & runtime to use
subscription_id="96aede12-2f73-41cb-b983-6d11a904839b"
resource_group_name="promptflow"
workspace_name="promptflow-canary"
runtime = "mir-0616"

In [ ]:
# Step 1: login & configure default workspace
#!az login

# set default workspace etc.
!az account set -s $subscription_id
!az configure --defaults group=$resource_group_name workspace=$workspace_name

In [ ]:
# Step 2: create custom connection if not exist in workspace
!pf connections create --name "custom_connection" --type Custom --env .env

In [ ]:
# Step 3: run bulk in cloud
!pf run_bulk --runtime $runtime --input ./data/denormalized-flat.jsonl --connection "custom_connection"

In [ ]:
# Step 3.1: wait for completion
# NOTE: Copy the bulk run id from the output of previous bulk run step, 
# e.g.: azureml://experiment/5fbfda62-4e3d-43da-b908-8b8feca82b17/flow/e943b457-acd4-4b4a-9d72-765bb81fe758/bulktest/b9a1c98a-87bd-4bce-bf1d-a392ffad321c/run/f88faee6-e510-45b7-9e63-08671b30b3a2
!pf stream "azureml://experiment/5fbfda62-4e3d-43da-b908-8b8feca82b17/flow/e943b457-acd4-4b4a-9d72-765bb81fe758/bulktest/b9a1c98a-87bd-4bce-bf1d-a392ffad321c/run/f88faee6-e510-45b7-9e63-08671b30b3a2"


In [ ]:
# Step 3.2: visualize run data
# NOTE: Copy the bulk run id from the output of previous bulk run step
!pf visualize "azureml://experiment/5fbfda62-4e3d-43da-b908-8b8feca82b17/flow/e943b457-acd4-4b4a-9d72-765bb81fe758/bulktest/b9a1c98a-87bd-4bce-bf1d-a392ffad321c/run/f88faee6-e510-45b7-9e63-08671b30b3a2"

In [ ]:
# Step 4: run evaluation in cloud
# NOTE: Copy the bulk run id from the output of previous bulk run step as variants
!pf eval --flow ../classification_accuracy_evaluation --runtime $runtime \
    --input ./data/denormalized-flat.jsonl --column-mapping "groundtruth=data.intent,prediction=variants.output.output" \
    --variants "azureml://experiment/5fbfda62-4e3d-43da-b908-8b8feca82b17/flow/e943b457-acd4-4b4a-9d72-765bb81fe758/bulktest/b9a1c98a-87bd-4bce-bf1d-a392ffad321c/run/f88faee6-e510-45b7-9e63-08671b30b3a2"

In [ ]:
# Step 4.1: wait for completion
# NOTE: Copy the evaluate run id from the output of previous step
!pf stream "azureml://experiment/5fbfda62-4e3d-43da-b908-8b8feca82b17/flow/e943b457-acd4-4b4a-9d72-765bb81fe758/bulktest/b9a1c98a-87bd-4bce-bf1d-a392ffad321c/run/dfacaf5d-769d-412a-8f88-cc4cce6e1eb2"


In [ ]:
# Step 4.2 visualize run data
# NOTE: Copy the evaluate flow id from the output of previous step
!pf visualize "azureml://experiment/5fbfda62-4e3d-43da-b908-8b8feca82b17/flow/e943b457-acd4-4b4a-9d72-765bb81fe758/bulktest/b9a1c98a-87bd-4bce-bf1d-a392ffad321c/run/dfacaf5d-769d-412a-8f88-cc4cce6e1eb2"

# Notes

The cli command demoed in this notebook is implemented here: 
https://msdata.visualstudio.com/DefaultCollection/Vienna/_git/PromptFlow?path=/src/promptflow-sdk/promptflow/_cli/pf.py


In [ ]:
import os
from promptflow._cli import pf
dir = os.path.dirname(pf.__file__)
print(dir)

In [ ]:
# copy the related files if you want to tweak some command.
!ls $dir